# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\UPSIT
df = pd.read_csv('UPSIT3Categ_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\UPSIT
(457, 5679)


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,UPSIT_Total,cg06067658,cg15442538,cg25928446,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,25,0.088679,0.890148,0.719238,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,200973410167_R04C01,3007,2,1,1,2,18,0.128668,0.916448,0.726893,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606
2,200973410167_R05C01,3010,2,1,1,2,9,0.106438,0.922987,0.715904,...,0.847166,0.880457,0.909545,0.902309,0.848555,0.881235,0.840847,0.886343,0.912116,0.786117


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','UPSIT_3','UPSIT_Total'], axis=1)
df1.head(2)

,APPRDX,cg06067658,cg15442538,cg25928446,cg12415575,cg26358225,cg19058526,cg20195763,cg23725394,cg27107076,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,1,0.088679,0.890148,0.719238,0.066678,0.151532,0.133249,0.479724,0.926745,0.538423,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,1,0.128668,0.916448,0.726893,0.066768,0.198886,0.141136,0.412058,0.876792,0.665097,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.48
2
Mean accuracy: 0.45999999999999996
3
Mean accuracy: 0.505
4
Mean accuracy: 0.5200000000000001
5
Mean accuracy: 0.5450000000000002
6
Mean accuracy: 0.545
7
Mean accuracy: 0.5649999999999998
8
Mean accuracy: 0.5499999999999999
9
Mean accuracy: 0.52
10
Mean accuracy: 0.44000000000000006
11
Mean accuracy: 0.47000000000000003
12
Mean accuracy: 0.4950000000000001
13
Mean accuracy: 0.5650000000000001
14
Mean accuracy: 0.475
15
Mean accuracy: 0.53
16
Mean accuracy: 0.55
17
Mean accuracy: 0.535
18
Mean accuracy: 0.51
19
Mean accuracy: 0.5
20
Mean accuracy: 0.545
21
Mean accuracy: 0.48500000000000004
22
Mean accuracy: 0.41500000000000004
23
Mean accuracy: 0.48
24
Mean accuracy: 0.425
25
Mean accuracy: 0.51
26
Mean accuracy: 0.525
27
Mean accuracy: 0.535
28
Mean accuracy: 0.48
29
Mean accuracy: 0.52
30
Mean accuracy: 0.475
31
Mean accuracy: 0.5599999999999999
32
Mean accuracy: 0.44000000000000006
33
Mean accuracy: 0.44000000000000006
34
Mean accuracy: 0.5
35
Mean accuracy:

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_AllCpG_100RF.csv", index = False)

0.50975


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.485
2
Mean accuracy: 0.5599999999999999
3
Mean accuracy: 0.51
4
Mean accuracy: 0.51
5
Mean accuracy: 0.46499999999999997
6
Mean accuracy: 0.4600000000000001
7
Mean accuracy: 0.545
8
Mean accuracy: 0.48500000000000004
9
Mean accuracy: 0.4600000000000001
10
Mean accuracy: 0.475
11
Mean accuracy: 0.49000000000000005
12
Mean accuracy: 0.525
13
Mean accuracy: 0.5349999999999999
14
Mean accuracy: 0.5650000000000001
15
Mean accuracy: 0.5149999999999999
16
Mean accuracy: 0.5549999999999999
17
Mean accuracy: 0.545
18
Mean accuracy: 0.475
19
Mean accuracy: 0.515
20
Mean accuracy: 0.51
21
Mean accuracy: 0.49000000000000005
22
Mean accuracy: 0.525
23
Mean accuracy: 0.535
24
Mean accuracy: 0.5599999999999999
25
Mean accuracy: 0.465
26
Mean accuracy: 0.475
27
Mean accuracy: 0.47000000000000003
28
Mean accuracy: 0.5650000000000001
29
Mean accuracy: 0.5349999999999999
30
Mean accuracy: 0.445
31
Mean accuracy: 0.53
32
Mean accuracy: 0.43000000000000005
33
Mean accuracy: 0.52999999999

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Rand50CpG_100RF.csv", index = False)

0.5103500000000001


## RanFor 100 times. Samples = 92

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(92, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.46499999999999997
2
Mean accuracy: 0.525
3
Mean accuracy: 0.62
4
Mean accuracy: 0.485
5
Mean accuracy: 0.5050000000000001
6
Mean accuracy: 0.5
7
Mean accuracy: 0.575
8
Mean accuracy: 0.53
9
Mean accuracy: 0.47000000000000003
10
Mean accuracy: 0.515
11
Mean accuracy: 0.5549999999999999
12
Mean accuracy: 0.51
13
Mean accuracy: 0.545
14
Mean accuracy: 0.6
15
Mean accuracy: 0.51
16
Mean accuracy: 0.545
17
Mean accuracy: 0.545
18
Mean accuracy: 0.445
19
Mean accuracy: 0.45499999999999996
20
Mean accuracy: 0.55
21
Mean accuracy: 0.53
22
Mean accuracy: 0.505
23
Mean accuracy: 0.5650000000000001
24
Mean accuracy: 0.5599999999999999
25
Mean accuracy: 0.48
26
Mean accuracy: 0.495
27
Mean accuracy: 0.5149999999999999
28
Mean accuracy: 0.5850000000000002
29
Mean accuracy: 0.4800000000000001
30
Mean accuracy: 0.51
31
Mean accuracy: 0.52
32
Mean accuracy: 0.45499999999999996
33
Mean accuracy: 0.5650000000000001
34
Mean accuracy: 0.5399999999999999
35
Mean accuracy: 0.4549999999999

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Rand92CpG_100RF.csv", index = False)

0.51095


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('UPSIT_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(457, 52)


,PATNO,APPRDX,cg14719675,cg00260798,cg10244356,cg00877792,cg03476938,cg10061292,cg23205529,cg23074747,...,cg27114120,cg06421138,cg01117061,cg08125401,cg07897734,cg06632027,cg00538060,cg05142211,cg13959611,cg19458274
0,3001,1,0.043587,0.885679,0.592379,0.497734,0.549004,0.013281,0.054117,0.011630,...,0.253900,0.891901,0.024752,0.136334,0.842938,0.029793,0.785801,0.035018,0.044262,0.018518
1,3007,1,0.080217,0.916535,0.609867,0.453499,0.561954,0.012520,0.048337,0.011863,...,0.251218,0.782571,0.080059,0.081183,0.923446,0.027356,0.818536,0.041906,0.033433,0.019884


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg14719675,cg00260798,cg10244356,cg00877792,cg03476938,cg10061292,cg23205529,cg23074747,cg02152968,...,cg27114120,cg06421138,cg01117061,cg08125401,cg07897734,cg06632027,cg00538060,cg05142211,cg13959611,cg19458274
0,1,0.043587,0.885679,0.592379,0.497734,0.549004,0.013281,0.054117,0.011630,0.263403,...,0.253900,0.891901,0.024752,0.136334,0.842938,0.029793,0.785801,0.035018,0.044262,0.018518
1,1,0.080217,0.916535,0.609867,0.453499,0.561954,0.012520,0.048337,0.011863,0.231128,...,0.251218,0.782571,0.080059,0.081183,0.923446,0.027356,0.818536,0.041906,0.033433,0.019884


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6449999999999999
2
Mean accuracy: 0.64
3
Mean accuracy: 0.66
4
Mean accuracy: 0.635
5
Mean accuracy: 0.7100000000000001
6
Mean accuracy: 0.6399999999999999
7
Mean accuracy: 0.6950000000000001
8
Mean accuracy: 0.635
9
Mean accuracy: 0.655
10
Mean accuracy: 0.665
11
Mean accuracy: 0.6500000000000001
12
Mean accuracy: 0.7250000000000001
13
Mean accuracy: 0.655
14
Mean accuracy: 0.5999999999999999
15
Mean accuracy: 0.655
16
Mean accuracy: 0.675
17
Mean accuracy: 0.7150000000000001
18
Mean accuracy: 0.6399999999999999
19
Mean accuracy: 0.625
20
Mean accuracy: 0.6599999999999999
21
Mean accuracy: 0.65
22
Mean accuracy: 0.58
23
Mean accuracy: 0.565
24
Mean accuracy: 0.625
25
Mean accuracy: 0.65
26
Mean accuracy: 0.6599999999999999
27
Mean accuracy: 0.63
28
Mean accuracy: 0.595
29
Mean accuracy: 0.68
30
Mean accuracy: 0.63
31
Mean accuracy: 0.655
32
Mean accuracy: 0.6449999999999999
33
Mean accuracy: 0.63
34
Mean accuracy: 0.63
35
Mean accuracy: 0.65
36
Mean accuracy: 0.7050

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Boruta50CpG_100RF.csv", index = False)

0.6503500000000001


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('UPSIT_NMI_Top50_Data.csv')
print(df.shape)
df.head(2)

(457, 51)


,APPRDX,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
0,1,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215
1,1,0.904315,0.021968,0.329010,0.805755,0.942176,0.810976,0.578553,0.225155,0.927726,...,0.925093,0.031999,0.915653,0.911271,0.045513,0.041906,0.017375,0.843436,0.033433,0.083898


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7
2
Mean accuracy: 0.62
3
Mean accuracy: 0.6799999999999999
4
Mean accuracy: 0.645
5
Mean accuracy: 0.695
6
Mean accuracy: 0.71
7
Mean accuracy: 0.59
8
Mean accuracy: 0.665
9
Mean accuracy: 0.67
10
Mean accuracy: 0.65
11
Mean accuracy: 0.6449999999999999
12
Mean accuracy: 0.625
13
Mean accuracy: 0.62
14
Mean accuracy: 0.67
15
Mean accuracy: 0.6649999999999999
16
Mean accuracy: 0.7150000000000001
17
Mean accuracy: 0.655
18
Mean accuracy: 0.6950000000000001
19
Mean accuracy: 0.6950000000000001
20
Mean accuracy: 0.6599999999999999
21
Mean accuracy: 0.6799999999999999
22
Mean accuracy: 0.59
23
Mean accuracy: 0.6399999999999999
24
Mean accuracy: 0.605
25
Mean accuracy: 0.645
26
Mean accuracy: 0.63
27
Mean accuracy: 0.7150000000000001
28
Mean accuracy: 0.6199999999999999
29
Mean accuracy: 0.6649999999999999
30
Mean accuracy: 0.625
31
Mean accuracy: 0.6500000000000001
32
Mean accuracy: 0.64
33
Mean accuracy: 0.64
34
Mean accuracy: 0.6249999999999999
35
Mean accuracy: 0.6849

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_NMI50CpG_100RF.csv", index = False)

0.6574000000000001


## RanFor Top 50 Boruta + NMI 100 times. Samples = 92

In [18]:
df = pd.read_csv('UPSIT_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(457, 94)


,PATNO,APPRDX,cg14719675,cg00260798,cg10244356,cg00877792,cg03476938,cg10061292,cg23205529,cg23074747,...,cg25314532,cg25557973,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg19731340,cg06794504,cg14149699
0,3001,1,0.043587,0.885679,0.592379,0.497734,0.549004,0.013281,0.054117,0.011630,...,0.013287,0.840012,0.930609,0.024186,0.884052,0.921355,0.025708,0.018411,0.752823,0.030215
1,3007,1,0.080217,0.916535,0.609867,0.453499,0.561954,0.012520,0.048337,0.011863,...,0.010308,0.786598,0.925093,0.031999,0.915653,0.911271,0.045513,0.017375,0.843436,0.083898


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg14719675,cg00260798,cg10244356,cg00877792,cg03476938,cg10061292,cg23205529,cg23074747,cg02152968,...,cg25314532,cg25557973,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg19731340,cg06794504,cg14149699
0,1,0.043587,0.885679,0.592379,0.497734,0.549004,0.013281,0.054117,0.011630,0.263403,...,0.013287,0.840012,0.930609,0.024186,0.884052,0.921355,0.025708,0.018411,0.752823,0.030215
1,1,0.080217,0.916535,0.609867,0.453499,0.561954,0.012520,0.048337,0.011863,0.231128,...,0.010308,0.786598,0.925093,0.031999,0.915653,0.911271,0.045513,0.017375,0.843436,0.083898


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7100000000000001
2
Mean accuracy: 0.615
3
Mean accuracy: 0.67
4
Mean accuracy: 0.635
5
Mean accuracy: 0.6900000000000002
6
Mean accuracy: 0.715
7
Mean accuracy: 0.725
8
Mean accuracy: 0.665
9
Mean accuracy: 0.64
10
Mean accuracy: 0.665
11
Mean accuracy: 0.6350000000000001
12
Mean accuracy: 0.685
13
Mean accuracy: 0.635
14
Mean accuracy: 0.6649999999999999
15
Mean accuracy: 0.68
16
Mean accuracy: 0.69
17
Mean accuracy: 0.69
18
Mean accuracy: 0.6399999999999999
19
Mean accuracy: 0.6600000000000001
20
Mean accuracy: 0.655
21
Mean accuracy: 0.67
22
Mean accuracy: 0.6100000000000001
23
Mean accuracy: 0.6599999999999999
24
Mean accuracy: 0.635
25
Mean accuracy: 0.6199999999999999
26
Mean accuracy: 0.655
27
Mean accuracy: 0.6450000000000001
28
Mean accuracy: 0.6500000000000001
29
Mean accuracy: 0.69
30
Mean accuracy: 0.635
31
Mean accuracy: 0.6950000000000001
32
Mean accuracy: 0.685
33
Mean accuracy: 0.655
34
Mean accuracy: 0.6050000000000001
35
Mean accuracy: 0.67
36
Mean 

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Top50sCpG92_100RF.csv", index = False)

0.6670500000000001
